# LEAPS Trade Readiness Analysis

This notebook deep-dives conviction tickers, selects high-quality call contracts across **three horizons** (Short <35 DTE, Medium 36-140 DTE, LEAPS 260+ DTE), computes Greeks, runs scenario stress tests, and produces an execution plan.

**Outputs**

- per-ticker LEAPS contract recommendation with Greeks and scenario P&L
- multi-horizon call comparison (short / medium / LEAPS) for each conviction name
- readiness checklist, deep-dive analytics, and a 6-position portfolio action plan


In [ ]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

from notebook_pipeline import (
    setup_report_style,
    parse_env_list,
    display_table,
    fetch_underlying_metrics,
    fetch_option_candidates,
    score_option_candidates,
    bs_call_greeks,
    scenario_pnl_call,
)

setup_report_style()

FIGURE_COUNTER = 0


def show_figure(fig, title):
    global FIGURE_COUNTER
    FIGURE_COUNTER += 1
    if not str(title).lower().startswith("figure"):
        title = f"Figure {FIGURE_COUNTER}. {title}"
    fig.update_layout(title=title)
    fig.show()


In [ ]:
CONVICTION_TICKERS = parse_env_list("CONVICTION_TICKERS", "GEV,CMI,KLAC,SNPS")
COMPARE_TICKERS = parse_env_list("COMPARE_TICKERS", "TSM,GOOG,AMAT,AVGO")

QUICK_RUN = os.getenv("QUICK_RUN", "0") == "1"
RATE_LIMIT_SLEEP = float(os.getenv("RATE_LIMIT_SLEEP", "0.20"))
STARTING_BALANCE = float(os.getenv("STARTING_BALANCE", "15000"))

if QUICK_RUN:
    CONVICTION_TICKERS = CONVICTION_TICKERS[:2]
    COMPARE_TICKERS = COMPARE_TICKERS[:2]
    RATE_LIMIT_SLEEP = min(RATE_LIMIT_SLEEP, 0.10)

# All three horizons — the LEAPS deep-dive uses the "leaps" bucket;
# short & medium are shown in the Multi-Horizon Opportunities section.
ALL_HORIZONS = {
    "short": {"min_dte": 7, "max_dte": 35, "target_dte": 21},
    "medium": {"min_dte": 36, "max_dte": 140, "target_dte": 75},
    "leaps": {"min_dte": 260, "max_dte": 760, "target_dte": 420},
}
ALL_CALL_MONEYNESS = {
    "short": (0.95, 1.08),
    "medium": (0.90, 1.12),
    "leaps": (0.72, 1.05),
}

# Backwards-compat aliases used by the LEAPS deep-dive cells
HORIZONS = {"leaps": ALL_HORIZONS["leaps"]}
LEAPS_MONEYNESS = {"leaps": ALL_CALL_MONEYNESS["leaps"]}

display(Markdown(f"**Conviction Tickers:** {', '.join(CONVICTION_TICKERS)}"))
display(Markdown(f"**Compare Tickers:** {', '.join(COMPARE_TICKERS)}"))


**Conviction Tickers:** GEV, CMI, KLAC, SNPS

**Compare Tickers:** TSM, GOOG, AMAT, AVGO

In [13]:
all_tickers = list(dict.fromkeys(CONVICTION_TICKERS + COMPARE_TICKERS))
metrics_df = fetch_underlying_metrics(
    all_tickers,
    history_period="1y",
    rate_limit_sleep=RATE_LIMIT_SLEEP,
)

focus_metrics = metrics_df[metrics_df["ticker"].isin(CONVICTION_TICKERS)].copy()
if focus_metrics.empty:
    raise RuntimeError("No conviction ticker metrics were fetched.")

call_frames = []
for _, row in focus_metrics.iterrows():
    calls = fetch_option_candidates(
        row["ticker"],
        side="call",
        spot=float(row["spot"]),
        horizons=HORIZONS,
        moneyness_bounds=LEAPS_MONEYNESS,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
        min_open_interest=15,
        min_volume=1,
    )
    if not calls.empty:
        call_frames.append(calls)

focus_chain_df = (
    pd.concat(call_frames, ignore_index=True) if call_frames else pd.DataFrame()
)
if focus_chain_df.empty:
    raise RuntimeError("No LEAPS contracts found for conviction tickers.")

focus_scored_df = score_option_candidates(focus_chain_df, focus_metrics)
best_calls = (
    focus_scored_df.sort_values("master_score", ascending=False)
    .groupby("ticker", as_index=False)
    .first()
    .sort_values("master_score", ascending=False)
)

readiness_rows = []
for _, row in best_calls.iterrows():
    greeks = bs_call_greeks(
        spot=float(row["spot"]),
        strike=float(row["strike"]),
        dte=int(row["dte"]),
        iv=float(row["iv"]),
        r=0.04,
    )
    scenarios = scenario_pnl_call(
        spot=float(row["spot"]),
        strike=float(row["strike"]),
        premium=float(row["mid"]),
        moves={"bear": -0.20, "base": 0.00, "bull": 0.20},
    )
    readiness_rows.append(
        {
            "ticker": row["ticker"],
            "contract_symbol": row["contract_symbol"],
            "dte": int(row["dte"]),
            "spot": float(row["spot"]),
            "strike": float(row["strike"]),
            "premium": float(row["mid"]),
            "iv": float(row["iv"]),
            "master_score": float(row["master_score"]),
            "delta": greeks["delta"],
            "gamma": greeks["gamma"],
            "theta": greeks["theta"],
            "vega": greeks["vega"],
            "pnl_bear": scenarios["bear"],
            "pnl_base": scenarios["base"],
            "pnl_bull": scenarios["bull"],
            "spread_pct": float(row["spread_pct"]),
            "open_interest": float(row["open_interest"]),
        }
    )

readiness_df = pd.DataFrame(readiness_rows)

checks = []
for _, row in readiness_df.iterrows():
    checks.append(
        {
            "ticker": row["ticker"],
            "liquidity_ok": row["open_interest"] >= 50 and row["spread_pct"] <= 0.25,
            "theta_ok": abs(row["theta"]) <= row["premium"] * 0.015,
            "iv_ok": row["iv"] <= 0.70,
            "bear_case_ok": row["pnl_bear"] >= -row["premium"],
        }
    )
check_df = pd.DataFrame(checks)
if not check_df.empty:
    check_df["checks_passed"] = check_df[
        ["liquidity_ok", "theta_ok", "iv_ok", "bear_case_ok"]
    ].sum(axis=1)

display_table(
    readiness_df[
        [
            "ticker",
            "contract_symbol",
            "dte",
            "spot",
            "strike",
            "premium",
            "iv",
            "master_score",
            "delta",
            "theta",
            "vega",
            "pnl_bear",
            "pnl_base",
            "pnl_bull",
        ]
    ],
    caption="Conviction LEAPS Trade Readiness",
    format_dict={
        "spot": "${:,.2f}",
        "strike": "${:,.2f}",
        "premium": "${:,.2f}",
        "iv": "{:.1%}",
        "master_score": "{:.1f}",
        "delta": "{:.3f}",
        "theta": "{:.3f}",
        "vega": "{:.3f}",
        "pnl_bear": "${:+.2f}",
        "pnl_base": "${:+.2f}",
        "pnl_bull": "${:+.2f}",
    },
)

if not check_df.empty:
    display_table(check_df, caption="Pre-Trade Checklist")


,ticker,contract_symbol,dte,spot,strike,premium,iv,master_score,delta,theta,vega,pnl_bear,pnl_base,pnl_bull
0,GEV,GEV270115C00670000,341,$779.35,$670.00,$219.25,57.5%,73.9,0.731,-0.247,2.484,$-219.25,$-109.90,$+45.97
1,KLAC,KLAC270115C01040000,341,"$1,442.95","$1,040.00",$470.50,45.6%,71.2,0.853,-0.295,3.213,$-356.14,$-67.55,$+221.04
2,SNPS,SNPS270115C00400000,341,$426.88,$400.00,$98.00,53.1%,37.0,0.676,-0.136,1.483,$-98.00,$-71.12,$+14.26


,ticker,liquidity_ok,theta_ok,iv_ok,bear_case_ok,checks_passed
0,GEV,True,True,True,True,4
1,KLAC,True,True,True,True,4
2,SNPS,True,True,True,True,4


In [14]:
if not readiness_df.empty:
    scenario_plot = readiness_df.melt(
        id_vars=["ticker"],
        value_vars=["pnl_bear", "pnl_base", "pnl_bull"],
        var_name="scenario",
        value_name="pnl",
    )
    fig_scen = px.bar(
        scenario_plot,
        x="ticker",
        y="pnl",
        color="scenario",
        barmode="group",
        color_discrete_sequence=["#B0533C", "#4C6E91", "#1F3A5F"],
        height=500,
    )
    show_figure(fig_scen, "Scenario P&L by Conviction Ticker")

    fig_greek = px.scatter(
        readiness_df,
        x="delta",
        y="vega",
        size="master_score",
        color="ticker",
        hover_data=["theta", "iv", "dte"],
        height=500,
    )
    show_figure(fig_greek, "Delta-Vega Position Map")


## Deep-Dive Analytics

Break-even distance, risk-reward profile, implied volatility comparison, Greeks fingerprint, and projected theta erosion for each conviction position.


In [ ]:
if not readiness_df.empty:
    rdf = readiness_df.copy()

    # --- 1. Break-Even Distance Waterfall ---
    rdf["breakeven"] = rdf["strike"] + rdf["premium"]
    rdf["breakeven_pct"] = rdf["breakeven"] / rdf["spot"] - 1
    fig_be = px.bar(
        rdf.sort_values("breakeven_pct"),
        x="ticker",
        y="breakeven_pct",
        color="breakeven_pct",
        color_continuous_scale="RdYlGn_r",
        height=480,
        text=rdf.sort_values("breakeven_pct")["breakeven_pct"].apply(
            lambda x: f"{x:+.1%}"
        ),
    )
    fig_be.update_traces(textposition="outside")
    fig_be.update_yaxes(tickformat=".0%", title="Break-Even Distance from Spot")
    fig_be.update_layout(coloraxis_showscale=False)
    show_figure(fig_be, "Break-Even Distance from Current Spot")

    # --- 2. Risk / Reward Profile (max loss vs bull P&L) ---
    fig_rr = go.Figure()
    fig_rr.add_trace(
        go.Bar(
            x=rdf["ticker"],
            y=-rdf["premium"],
            name="Max Loss (Premium)",
            marker_color="#B0533C",
        )
    )
    fig_rr.add_trace(
        go.Bar(
            x=rdf["ticker"],
            y=rdf["pnl_bull"],
            name="Bull Case P&L (+20%)",
            marker_color="#1F3A5F",
        )
    )
    fig_rr.add_trace(
        go.Bar(
            x=rdf["ticker"],
            y=rdf["pnl_base"],
            name="Base Case P&L (flat)",
            marker_color="#4C6E91",
        )
    )
    fig_rr.update_layout(
        barmode="group", height=500, yaxis_title="P&L per Contract ($)"
    )
    show_figure(fig_rr, "Risk-Reward Profile by Position")

    # --- 3. IV Comparison Lollipop ---
    rdf_sorted = rdf.sort_values("iv", ascending=True)
    fig_iv = go.Figure()
    fig_iv.add_trace(
        go.Scatter(
            x=rdf_sorted["iv"],
            y=rdf_sorted["ticker"],
            mode="markers",
            marker=dict(size=14, color="#1F3A5F"),
        )
    )
    for _, r in rdf_sorted.iterrows():
        fig_iv.add_shape(
            type="line",
            x0=0,
            x1=r["iv"],
            y0=r["ticker"],
            y1=r["ticker"],
            line=dict(color="#4C6E91", width=2),
        )
    fig_iv.update_xaxes(tickformat=".0%", title="Implied Volatility")
    fig_iv.update_layout(height=420, showlegend=False)
    show_figure(fig_iv, "Implied Volatility Comparison")

    # --- 4. Greeks Radar (normalized to max) ---
    greek_cols = ["delta", "gamma", "vega"]
    for col in greek_cols:
        mx = rdf[col].abs().max()
        rdf[f"{col}_n"] = rdf[col].abs() / mx if mx > 0 else 0
    # add theta as absolute (lower is better, so invert)
    mx_theta = rdf["theta"].abs().max()
    rdf["theta_n"] = 1 - (rdf["theta"].abs() / mx_theta) if mx_theta > 0 else 0

    fig_radar = go.Figure()
    cats = ["Delta", "Gamma", "Vega", "Theta (inverted)"]
    for _, r in rdf.iterrows():
        vals = [r["delta_n"], r["gamma_n"], r["vega_n"], r["theta_n"]]
        fig_radar.add_trace(
            go.Scatterpolar(
                r=vals + [vals[0]],
                theta=cats + [cats[0]],
                fill="toself",
                name=r["ticker"],
                opacity=0.55,
            )
        )
    fig_radar.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 1.05])),
        height=520,
    )
    show_figure(fig_radar, "Greeks Fingerprint (Normalized)")

    # --- 5. Theta Decay Projection (30/60/90/180 day erosion) ---
    decay_rows = []
    for _, r in rdf.iterrows():
        daily_theta = abs(r["theta"])
        for days in [30, 60, 90, 120, 180]:
            eroded = daily_theta * days
            remaining = max(r["premium"] - eroded, 0)
            decay_rows.append(
                {
                    "ticker": r["ticker"],
                    "days": days,
                    "value_remaining": remaining,
                    "pct_remaining": remaining / r["premium"]
                    if r["premium"] > 0
                    else 0,
                }
            )
    decay_df = pd.DataFrame(decay_rows)
    fig_decay = px.line(
        decay_df,
        x="days",
        y="pct_remaining",
        color="ticker",
        markers=True,
        height=480,
    )
    fig_decay.update_yaxes(tickformat=".0%", title="Premium Remaining")
    fig_decay.update_xaxes(title="Days Held")
    fig_decay.add_hline(
        y=0.50, line_dash="dash", line_color="gray", annotation_text="50% erosion"
    )
    show_figure(fig_decay, "Projected Theta Erosion Over Time")


## Multi-Horizon Call Opportunities

Best call contract at each horizon (Short <35 DTE, Medium 36-140 DTE, LEAPS 260+ DTE) for every conviction ticker. Use this to compare near-term income plays against long-dated conviction bets side by side.


In [ ]:
# Fetch calls across ALL three horizons for conviction tickers
multi_frames = []
for _, row in focus_metrics.iterrows():
    mh_calls = fetch_option_candidates(
        row["ticker"],
        side="call",
        spot=float(row["spot"]),
        horizons=ALL_HORIZONS,
        moneyness_bounds=ALL_CALL_MONEYNESS,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
        min_open_interest=5,
        min_volume=1,
    )
    if not mh_calls.empty:
        multi_frames.append(mh_calls)

multi_chain_df = (
    pd.concat(multi_frames, ignore_index=True) if multi_frames else pd.DataFrame()
)

if multi_chain_df.empty:
    display(Markdown("*No multi-horizon call data returned.*"))
else:
    multi_scored = score_option_candidates(multi_chain_df, focus_metrics)

    # Best call per ticker per horizon
    best_per_horizon = (
        multi_scored.sort_values("master_score", ascending=False)
        .groupby(["ticker", "horizon"], as_index=False)
        .first()
        .sort_values(["ticker", "horizon"])
    )

    # --- Comparison Table ---
    show_cols = [
        "ticker",
        "horizon",
        "contract_symbol",
        "dte",
        "strike",
        "mid",
        "iv",
        "expected_return",
        "master_score",
    ]
    avail_cols = [c for c in show_cols if c in best_per_horizon.columns]
    display_table(
        best_per_horizon[avail_cols],
        caption="Best Call per Ticker and Horizon",
        format_dict={
            "strike": "${:,.2f}",
            "mid": "${:,.2f}",
            "iv": "{:.1%}",
            "expected_return": "{:.1%}",
            "master_score": "{:.1f}",
        },
    )

    # --- 1. Grouped Bar: Score by Horizon per Ticker ---
    fig_hz_score = px.bar(
        best_per_horizon.sort_values("master_score", ascending=True),
        x="master_score",
        y="ticker",
        color="horizon",
        orientation="h",
        barmode="group",
        hover_data=["dte", "strike", "mid", "iv"],
        height=max(400, len(CONVICTION_TICKERS) * 80),
        color_discrete_map={
            "short": "#B0533C",
            "medium": "#4C6E91",
            "leaps": "#1F3A5F",
        },
        category_orders={"horizon": ["short", "medium", "leaps"]},
    )
    show_figure(fig_hz_score, "Master Score by Horizon per Ticker")

    # --- 2. IV across horizons ---
    fig_hz_iv = px.bar(
        best_per_horizon.sort_values(["ticker", "horizon"]),
        x="ticker",
        y="iv",
        color="horizon",
        barmode="group",
        height=460,
        color_discrete_map={
            "short": "#B0533C",
            "medium": "#4C6E91",
            "leaps": "#1F3A5F",
        },
        category_orders={"horizon": ["short", "medium", "leaps"]},
    )
    fig_hz_iv.update_yaxes(tickformat=".0%", title="Implied Volatility")
    show_figure(fig_hz_iv, "IV Term Structure by Ticker")

    # --- 3. Premium vs DTE scatter ---
    fig_hz_prem = px.scatter(
        best_per_horizon,
        x="dte",
        y="mid",
        color="ticker",
        symbol="horizon",
        size="master_score",
        hover_data=["contract_symbol", "strike", "iv"],
        height=500,
    )
    fig_hz_prem.update_yaxes(title="Premium ($)", tickprefix="$")
    fig_hz_prem.update_xaxes(title="Days to Expiration")
    show_figure(fig_hz_prem, "Premium vs DTE Landscape")

    # --- 4. Expected Return comparison ---
    if "expected_return" in best_per_horizon.columns:
        fig_hz_ret = px.bar(
            best_per_horizon.sort_values(["ticker", "horizon"]),
            x="ticker",
            y="expected_return",
            color="horizon",
            barmode="group",
            height=460,
            color_discrete_map={
                "short": "#B0533C",
                "medium": "#4C6E91",
                "leaps": "#1F3A5F",
            },
            category_orders={"horizon": ["short", "medium", "leaps"]},
        )
        fig_hz_ret.update_yaxes(tickformat=".0%", title="Expected Return")
        show_figure(fig_hz_ret, "Expected Return by Horizon")

    # --- 5. Horizon coverage heatmap ---
    pivot = best_per_horizon.pivot_table(
        index="ticker", columns="horizon", values="master_score", aggfunc="first"
    )
    for h in ["short", "medium", "leaps"]:
        if h not in pivot.columns:
            pivot[h] = np.nan
    pivot = pivot[["short", "medium", "leaps"]]

    fig_heat = go.Figure(
        data=go.Heatmap(
            z=pivot.values,
            x=["Short (<35d)", "Medium (36-140d)", "LEAPS (260d+)"],
            y=pivot.index.tolist(),
            colorscale="Blues",
            text=np.where(
                np.isnan(pivot.values),
                "no chain",
                np.round(pivot.values, 1).astype(str),
            ),
            texttemplate="%{text}",
            hovertemplate="Ticker: %{y}<br>Horizon: %{x}<br>Score: %{z:.1f}<extra></extra>",
        )
    )
    fig_heat.update_layout(height=max(350, len(pivot) * 60))
    show_figure(fig_heat, "Horizon Coverage Heatmap (Score)")

    # --- Quick-glance summary ---
    display(Markdown("### Multi-Horizon Takeaway"))
    for tkr in CONVICTION_TICKERS:
        tkr_rows = best_per_horizon[best_per_horizon["ticker"] == tkr]
        if tkr_rows.empty:
            display(Markdown(f"- **{tkr}**: no calls found across any horizon"))
            continue
        parts = []
        for _, r in tkr_rows.sort_values("dte").iterrows():
            parts.append(
                f"{r['horizon'].capitalize()} {int(r['dte'])}d "
                f"${r['strike']:,.0f}C @${r['mid']:,.2f} "
                f"(score {r['master_score']:.1f})"
            )
        display(Markdown(f"- **{tkr}**: " + " | ".join(parts)))


## Portfolio Action Plan

Score-weighted position sizing for a concentrated LEAPS portfolio. Positions are capped at six names and sized proportionally to their master score, subject to a per-position floor and ceiling.


In [ ]:
MAX_POSITIONS = 6

if not readiness_df.empty:
    port = (
        readiness_df.sort_values("master_score", ascending=False)
        .head(MAX_POSITIONS)
        .copy()
    )

    # Score-weighted allocation
    total_score = port["master_score"].sum()
    port["weight"] = (
        port["master_score"] / total_score if total_score > 0 else 1 / len(port)
    )

    # Floor 5%, cap 35%, re-normalize
    port["weight"] = port["weight"].clip(lower=0.05, upper=0.35)
    port["weight"] = port["weight"] / port["weight"].sum()

    port["dollar_alloc"] = port["weight"] * STARTING_BALANCE
    port["contracts"] = (
        np.floor(port["dollar_alloc"] / (port["premium"] * 100))
        .astype(int)
        .clip(lower=1)
    )
    port["cost"] = port["contracts"] * port["premium"] * 100
    port["pct_of_capital"] = port["cost"] / STARTING_BALANCE

    # --- Allocation Donut ---
    fig_donut = go.Figure(
        data=[
            go.Pie(
                labels=port["ticker"],
                values=port["cost"],
                hole=0.50,
                marker_colors=[
                    "#1F3A5F",
                    "#4C6E91",
                    "#8B9BB4",
                    "#B0533C",
                    "#D4A574",
                    "#6B8E6B",
                ],
                textinfo="label+percent",
            )
        ]
    )
    fig_donut.update_layout(
        height=480,
        annotations=[
            dict(
                text=f"${port['cost'].sum():,.0f}",
                x=0.5,
                y=0.5,
                font_size=18,
                showarrow=False,
            )
        ],
    )
    show_figure(fig_donut, "Portfolio Allocation")

    # --- Action Table ---
    display_table(
        port[
            [
                "ticker",
                "contract_symbol",
                "strike",
                "premium",
                "dte",
                "contracts",
                "cost",
                "pct_of_capital",
                "master_score",
            ]
        ],
        caption=f"Execution Plan ({MAX_POSITIONS} Positions, ${STARTING_BALANCE:,.0f} Capital)",
        format_dict={
            "strike": "${:,.2f}",
            "premium": "${:,.2f}",
            "contracts": "{:,.0f}",
            "cost": "${:,.0f}",
            "pct_of_capital": "{:.1%}",
            "master_score": "{:.1f}",
        },
    )

    # --- Plain-text action items ---
    display(Markdown("### Action Items"))
    lines = []
    for _, r in port.iterrows():
        lines.append(
            f"- **{r['ticker']}**: Buy **{int(r['contracts'])}** contract(s) of "
            f"`{r['contract_symbol']}` (${r['strike']:,.0f} strike, {int(r['dte'])}d) "
            f"at ~${r['premium']:,.2f} — cost ${r['cost']:,.0f} "
            f"({r['pct_of_capital']:.1%} of capital)"
        )
    lines.append(
        f"\n**Total deployed:** ${port['cost'].sum():,.0f} of ${STARTING_BALANCE:,.0f} "
        f"({port['cost'].sum() / STARTING_BALANCE:.1%})"
    )
    remaining = STARTING_BALANCE - port["cost"].sum()
    if remaining > 0:
        lines.append(
            f"**Cash reserve:** ${remaining:,.0f} ({remaining / STARTING_BALANCE:.1%})"
        )
    display(Markdown("\n".join(lines)))

    # --- Portfolio-level risk summary ---
    display(Markdown("### Portfolio Risk Summary"))
    total_max_loss = port["cost"].sum()
    total_bull_pnl = (port["pnl_bull"] * port["contracts"]).sum()
    avg_delta = (port["delta"] * port["contracts"]).sum() / port["contracts"].sum()
    avg_theta = (port["theta"] * port["contracts"]).sum()
    risk_lines = [
        f"- **Max loss (all premiums):** ${total_max_loss:,.0f}",
        f"- **Bull-case total P&L (+20%):** ${total_bull_pnl:+,.0f}",
        f"- **Weighted avg delta:** {avg_delta:.3f}",
        f"- **Daily portfolio theta:** ${avg_theta:+.2f}",
        f"- **Positions:** {len(port)}",
    ]
    if not check_df.empty:
        merged = port.merge(
            check_df[["ticker", "checks_passed"]], on="ticker", how="left"
        )
        all_pass = (merged["checks_passed"] == 4).sum()
        risk_lines.append(
            f"- **Fully passing pre-trade checks:** {all_pass} of {len(port)}"
        )
    display(Markdown("\n".join(risk_lines)))
else:
    display(Markdown("No readiness data available to build a portfolio plan."))
